# Notebook 3:
writing your own library like tensorflow, A MiniFlow library.

Before we start using tensorflow, keras or anyother library, it is best to learn how this library works conceptually. So, to do that we will write our own library - MiniFlow which will work similiarly like tensorFlow. Why this is important? Well, before using this abstractions Isn't it is good to learn how under the hood this library works? forward pass, backprop, derivatives or chain rule?

An intellectually curious mind should know this nutss and bolts and that is why we will write our first Neural Network in from scratch.

## MiniFlow Architecture:

Let's consider how to implement this graph structure in MiniFlow. We'll use a Python class to represent a generic node.

We know that each node might receive input from multiple other nodes. We also know that each node creates a single output, which will likely be passed to other nodes. Let's add two lists: one to store references to the inbound nodes, and the other to store references to the outbound nodes.

In [1]:
class Node(object):
    def __init__(self, inbound_nodes = []):
        #Nodes from which this Node recieves values
        self.inbound_nodes = inbound_nodes
        #Nodes to which this node pass values
        self.outbound_nodes = []
        #for each inbound_nodes add this Node as outbound Node.
        for inbound_node in self.inbound_nodes:
            inbound_node.outbound_nodes.append(self)
        #A calculated final value of this Node
        self.value = None
        
    def forward(self):
        '''
        Forward propagation.
        
        compute the output value based on 'inbound_nodes' and 
        store the result in self.value
        '''
        raise NotImplemented

*While Node defines the basic set of properties that every node holds, only specialized subclasses of will end uo in final graph. So, lets build our first subclass which will calculate the value and hold value.*

In [2]:
class Input(Node):
    def __init__(self):
        #An Input Node has no inbound nodes,
        #so no need to pass anything to the Node instantiator.
        Node.__init__(self)
    
    def forward(self, value=None):
        '''
        since Input node is the node which doesn't have any inbound_nodes
        this forward method will take value as input and set it self.value
        while, other non input Node's forward method will read the value form each inbound_nodes.value
        calculate the resultant and store it in self.value.
        '''
        #overwite the value if one passed in.
        if value is not None:
            self.value = value
        return self.value

*Okay, so far we wrote Input Node which doesn't take any input from other nodes. But itself holds the input to neural network and this value can be set directly by setting Node.vlaue or by passing value to forward method.*

*As we know, in nueral networks there are nodes which takes value from such input nodes or hidden nodes perform actual calculation and save it along with passing it to rest of network.*

*Okay, Lets implement Add Node which will does exactly this.*

In [3]:
class Add(Node):
    def __init__(self, inbound_nodes = []):
        Node.__init__(self, inbound_nodes)
    
    def forward(self):
        """
        Note: this method doesn't has value parameter as we supposed to take
        values from inbound_nodes and perform calculation.
        """
        self.value = 0
        for inbound_node in self.inbound_nodes:
                self.value += inbound_node.value
        
        return self.value

## Forward Propagation:
Like in tensorFlow library, we has to create the computation graph first which gets initialized when we pass values to it and call evaluate. It then checks the computation graph, runs through the computation and give us the output. Similiary, we will implement to menthods in the this library.

`topological_sort() and forward_pass()`, In order to define the network we need to define the order of operations on nodes. Given the input to some node depends on the output of others, we need to flatten this computation graph in such a way that all the nodes gets evaluated first whose inputs are needed to calculate the output of current node.

To resolve this we will implement kahn's Algorithm which will sort the nodes inthe order of their calculation. The input of `topological_sort()` is `feed_dict: a python dict` and the output is `sorted list of nodes`.

Then `forward_pass()` will take this `sorted_nodes` list and do a forward pass on each node and gives back the final `output_node` which will contain the final value of the network.

In [4]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

In [5]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

In [6]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""

x, y, z = Input(), Input(), Input()

f = Add([x, y, z])

feed_dict = {x: 10, y: 5, z:8}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort sets the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))

10 + 5 = 23 (according to miniflow)


Congratualtions!, on building your first feed forward nueral network.
Next this will be to compare output value:`y'` with true value:`y`, calculate error term and do backprop to adjust the weights to improve the model.


## Learning and loss

Cool, So far we implemented forward pass of neural network which outputs y'. The real learning starts after this in NN, the error term is calculated and backpropagation happen where the network weights and bias are adjusted using chain rule. This weights and bias are updated by a simple formula: W - alpha * del_of_errorterm_wrt_W, where alpha is learning rate.

To understand the learning happening in the network, let's implement `Linear` node which takes weight, bias and input(x) and outputs `$\sum(W_i*X_i) + bias$`

In [7]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.
        
        """
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        self.value = bias
        
        print("inbound_nodes[0].value: "+ str(inputs))
        print("inbound_nodes[1].value: "+ str(weights))
        print("inbound_nodes[2].value: "+ str(bias))
        for x, w in zip(inputs, weights):
            self.value += x * w


In [8]:
inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {
    inputs: [6, 14, 3],
    weights: [0.5, 0.25, 1.4],
    bias: 2
}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output) # should be 12.7 with this example

inbound_nodes[0].value: [6, 14, 3]
inbound_nodes[1].value: [0.5, 0.25, 1.4]
inbound_nodes[2].value: 2
12.7


In [9]:
"""
Modify Linear#forward so that it linearly transforms
input matrices, weights matrices and a bias vector to
an output.
"""
class Linear(Node):
    def __init__(self, X, W, b):
        # Notice the ordering of the input nodes passed to the
        # Node constructor.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Set the value of this node to the linear transform output.

        Your code goes here!
        """
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        
#         print("inbound_nodes[0].value: "+ str(inputs))
#         print("inbound_nodes[1].value: "+ str(weights))
#         print("inbound_nodes[2].value: "+ str(bias))
        self.value = np.dot(inputs, weights) + bias


In [10]:
"""
The setup is similar to the prevous `Linear` node you wrote
except you're now using NumPy arrays instead of python lists.

Update the Linear class in miniflow.py to work with
numpy vectors (arrays) and matrices.

Test your code here!
"""

import numpy as np

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)

[[-9.  4.]
 [-9.  4.]]


## Activation function: Sigmoid function


In [11]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used later with `backward` as well.

        `x`: A numpy array-like object.

        Return the result of the sigmoid function.

        Your code here!
        """
        return 1./(1 + np.exp(-x))


    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function, `_sigmoid`.

        Your code here!
        """
        # This is a dummy value to prevent numpy errors
        # if you test without changing this method.
        inputs = self.inbound_nodes[0].value
                
        self.value = self._sigmoid(inputs)


In [12]:
"""
This network feeds the output of a linear transform
to the sigmoid function.

Finish implementing the Sigmoid class in miniflow.py!

Feel free to play around with this network, too!
"""

import numpy as np

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

[[1.23394576e-04 9.82013790e-01]
 [1.23394576e-04 9.82013790e-01]]


## Error term or Cost Function
### MSE

In [13]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        # TODO: your code here
        #self.value = np.divide(np.sum(np.square(np.subtract(y, a))),y.shape[0])
        self.value = np.mean(np.square(np.subtract(y, a)))


In [14]:
import numpy as np

y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(cost, graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.416666666666668


## Backpropagation : way to calculate amount of change in error cost due to wach weight and bias
### Gradient Descent : way to adjust weights and bias

Great! We've successfully calculated a full forward pass and found the cost. Next we need to start a backwards pass, which starts with backpropagation. Backpropagation is the process by which the network runs error values backwards.

During this process, the network calculates the way in which the weights need to change (also called the gradient) to reduce the overall error of the network. Changing the weights usually occurs through a technique called gradient descent.

Making sense of the purpose of backpropagation comes more easily after you work through the intended outcome. I'll come back to backpropagation in a bit, but first, I want to dive deeper into gradient descent.

![alt text](images/SGD.png "Point in 3-d surface")

#### Intution behind backpropagation
Imagine a point on a surface in three dimensional space. In real-life, a ball sitting on the slope of a valley makes a nice analogy. In this case, the height of the point represents the difference between the current output of the network and the correct output given the current parameter values (hence why you need data with known outputs). Each dimension of the plane represents another parameter to the network. A network with m parameters would be a hypersurface of m dimensions.

(Imagining more than three dimensions is tricky. The good news is that the ball and valley example describes the behavior of gradient descent well, the only difference between three dimensional and n dimensional situations being the number of parameters in the calculations.)

In the ideal situation, the ball rests at the bottom of the valley, indicating the minimum difference between the output of the network and the known correct output.

The learning process starts with random weights and biases. In the ball analogy, the ball starts at a random point near the valley.

Gradient descent works by first calculating the slope of the plane at the current point, which includes calculating the partial derivatives of the loss with respect to all of the weights. This set of partial derivatives is called the gradient. Then it uses the gradient to modify the weights such that the next forward pass through the network moves the output lower in the hypersurface. Physically, this would be the same as measuring the slope of the valley at the location of the ball, and then moving the ball a small amount in the direction of the slope. Over time, it's possible to find the bottom of the valley with many small movements.

In [40]:
def gradient_descent_update(x, gradx, learning_rate):
    """
    Performs a gradient descent update.
    """
    # TODO: Implement gradient descent.
    
    # Return the new value for x
    return x - learning_rate * gradx

In [56]:
"""
Given the starting point of any `x` gradient descent
should be able to find the minimum value of x for the
cost function `f` defined below.
"""
import random


def f(x):
    """
    Quadratic function.

    It's easy to see the minimum value of the function
    is 5 when is x=0.
    """
    return x**2 + 5


def df(x):
    """
    Derivative of `f` with respect to `x`.
    """
    return 2*x


# Random number between 0 and 10,000. Feel free to set x whatever you like.
x = random.randint(0, 1000)
# TODO: Set the learning rate
learning_rate = 0.01

epochs = 10000

for i in range(epochs+1):
    cost = f(x)
    gradx = df(x)
    print("EPOCH {}: Cost = {:.3f}, x = {:.3f}".format(i, cost, gradx))
    x = gradient_descent_update(x, gradx, learning_rate)

EPOCH 0: Cost = 224681.000, x = 948.000
EPOCH 1: Cost = 215783.830, x = 929.040
EPOCH 2: Cost = 207238.989, x = 910.459
EPOCH 3: Cost = 199032.523, x = 892.250
EPOCH 4: Cost = 191151.033, x = 874.405
EPOCH 5: Cost = 183581.650, x = 856.917
EPOCH 6: Cost = 176312.015, x = 839.779
EPOCH 7: Cost = 169330.257, x = 822.983
EPOCH 8: Cost = 162624.977, x = 806.523
EPOCH 9: Cost = 156185.226, x = 790.393
EPOCH 10: Cost = 150000.489, x = 774.585
EPOCH 11: Cost = 144060.667, x = 759.093
EPOCH 12: Cost = 138356.063, x = 743.911
EPOCH 13: Cost = 132877.361, x = 729.033
EPOCH 14: Cost = 127615.615, x = 714.453
EPOCH 15: Cost = 122562.235, x = 700.164
EPOCH 16: Cost = 117708.968, x = 686.160
EPOCH 17: Cost = 113047.891, x = 672.437
EPOCH 18: Cost = 108571.393, x = 658.988
EPOCH 19: Cost = 104272.164, x = 645.809
EPOCH 20: Cost = 100143.184, x = 632.892
EPOCH 21: Cost = 96177.712, x = 620.235
EPOCH 22: Cost = 92369.272, x = 607.830
EPOCH 23: Cost = 88711.647, x = 595.673
EPOCH 24: Cost = 85198.864, x

EPOCH 1852: Cost = 5.000, x = 0.000
EPOCH 1853: Cost = 5.000, x = 0.000
EPOCH 1854: Cost = 5.000, x = 0.000
EPOCH 1855: Cost = 5.000, x = 0.000
EPOCH 1856: Cost = 5.000, x = 0.000
EPOCH 1857: Cost = 5.000, x = 0.000
EPOCH 1858: Cost = 5.000, x = 0.000
EPOCH 1859: Cost = 5.000, x = 0.000
EPOCH 1860: Cost = 5.000, x = 0.000
EPOCH 1861: Cost = 5.000, x = 0.000
EPOCH 1862: Cost = 5.000, x = 0.000
EPOCH 1863: Cost = 5.000, x = 0.000
EPOCH 1864: Cost = 5.000, x = 0.000
EPOCH 1865: Cost = 5.000, x = 0.000
EPOCH 1866: Cost = 5.000, x = 0.000
EPOCH 1867: Cost = 5.000, x = 0.000
EPOCH 1868: Cost = 5.000, x = 0.000
EPOCH 1869: Cost = 5.000, x = 0.000
EPOCH 1870: Cost = 5.000, x = 0.000
EPOCH 1871: Cost = 5.000, x = 0.000
EPOCH 1872: Cost = 5.000, x = 0.000
EPOCH 1873: Cost = 5.000, x = 0.000
EPOCH 1874: Cost = 5.000, x = 0.000
EPOCH 1875: Cost = 5.000, x = 0.000
EPOCH 1876: Cost = 5.000, x = 0.000
EPOCH 1877: Cost = 5.000, x = 0.000
EPOCH 1878: Cost = 5.000, x = 0.000
EPOCH 1879: Cost = 5.000, x 

EPOCH 3352: Cost = 5.000, x = 0.000
EPOCH 3353: Cost = 5.000, x = 0.000
EPOCH 3354: Cost = 5.000, x = 0.000
EPOCH 3355: Cost = 5.000, x = 0.000
EPOCH 3356: Cost = 5.000, x = 0.000
EPOCH 3357: Cost = 5.000, x = 0.000
EPOCH 3358: Cost = 5.000, x = 0.000
EPOCH 3359: Cost = 5.000, x = 0.000
EPOCH 3360: Cost = 5.000, x = 0.000
EPOCH 3361: Cost = 5.000, x = 0.000
EPOCH 3362: Cost = 5.000, x = 0.000
EPOCH 3363: Cost = 5.000, x = 0.000
EPOCH 3364: Cost = 5.000, x = 0.000
EPOCH 3365: Cost = 5.000, x = 0.000
EPOCH 3366: Cost = 5.000, x = 0.000
EPOCH 3367: Cost = 5.000, x = 0.000
EPOCH 3368: Cost = 5.000, x = 0.000
EPOCH 3369: Cost = 5.000, x = 0.000
EPOCH 3370: Cost = 5.000, x = 0.000
EPOCH 3371: Cost = 5.000, x = 0.000
EPOCH 3372: Cost = 5.000, x = 0.000
EPOCH 3373: Cost = 5.000, x = 0.000
EPOCH 3374: Cost = 5.000, x = 0.000
EPOCH 3375: Cost = 5.000, x = 0.000
EPOCH 3376: Cost = 5.000, x = 0.000
EPOCH 3377: Cost = 5.000, x = 0.000
EPOCH 3378: Cost = 5.000, x = 0.000
EPOCH 3379: Cost = 5.000, x 

EPOCH 4851: Cost = 5.000, x = 0.000
EPOCH 4852: Cost = 5.000, x = 0.000
EPOCH 4853: Cost = 5.000, x = 0.000
EPOCH 4854: Cost = 5.000, x = 0.000
EPOCH 4855: Cost = 5.000, x = 0.000
EPOCH 4856: Cost = 5.000, x = 0.000
EPOCH 4857: Cost = 5.000, x = 0.000
EPOCH 4858: Cost = 5.000, x = 0.000
EPOCH 4859: Cost = 5.000, x = 0.000
EPOCH 4860: Cost = 5.000, x = 0.000
EPOCH 4861: Cost = 5.000, x = 0.000
EPOCH 4862: Cost = 5.000, x = 0.000
EPOCH 4863: Cost = 5.000, x = 0.000
EPOCH 4864: Cost = 5.000, x = 0.000
EPOCH 4865: Cost = 5.000, x = 0.000
EPOCH 4866: Cost = 5.000, x = 0.000
EPOCH 4867: Cost = 5.000, x = 0.000
EPOCH 4868: Cost = 5.000, x = 0.000
EPOCH 4869: Cost = 5.000, x = 0.000
EPOCH 4870: Cost = 5.000, x = 0.000
EPOCH 4871: Cost = 5.000, x = 0.000
EPOCH 4872: Cost = 5.000, x = 0.000
EPOCH 4873: Cost = 5.000, x = 0.000
EPOCH 4874: Cost = 5.000, x = 0.000
EPOCH 4875: Cost = 5.000, x = 0.000
EPOCH 4876: Cost = 5.000, x = 0.000
EPOCH 4877: Cost = 5.000, x = 0.000
EPOCH 4878: Cost = 5.000, x 

EPOCH 6644: Cost = 5.000, x = 0.000
EPOCH 6645: Cost = 5.000, x = 0.000
EPOCH 6646: Cost = 5.000, x = 0.000
EPOCH 6647: Cost = 5.000, x = 0.000
EPOCH 6648: Cost = 5.000, x = 0.000
EPOCH 6649: Cost = 5.000, x = 0.000
EPOCH 6650: Cost = 5.000, x = 0.000
EPOCH 6651: Cost = 5.000, x = 0.000
EPOCH 6652: Cost = 5.000, x = 0.000
EPOCH 6653: Cost = 5.000, x = 0.000
EPOCH 6654: Cost = 5.000, x = 0.000
EPOCH 6655: Cost = 5.000, x = 0.000
EPOCH 6656: Cost = 5.000, x = 0.000
EPOCH 6657: Cost = 5.000, x = 0.000
EPOCH 6658: Cost = 5.000, x = 0.000
EPOCH 6659: Cost = 5.000, x = 0.000
EPOCH 6660: Cost = 5.000, x = 0.000
EPOCH 6661: Cost = 5.000, x = 0.000
EPOCH 6662: Cost = 5.000, x = 0.000
EPOCH 6663: Cost = 5.000, x = 0.000
EPOCH 6664: Cost = 5.000, x = 0.000
EPOCH 6665: Cost = 5.000, x = 0.000
EPOCH 6666: Cost = 5.000, x = 0.000
EPOCH 6667: Cost = 5.000, x = 0.000
EPOCH 6668: Cost = 5.000, x = 0.000
EPOCH 6669: Cost = 5.000, x = 0.000
EPOCH 6670: Cost = 5.000, x = 0.000
EPOCH 6671: Cost = 5.000, x 

EPOCH 8350: Cost = 5.000, x = 0.000
EPOCH 8351: Cost = 5.000, x = 0.000
EPOCH 8352: Cost = 5.000, x = 0.000
EPOCH 8353: Cost = 5.000, x = 0.000
EPOCH 8354: Cost = 5.000, x = 0.000
EPOCH 8355: Cost = 5.000, x = 0.000
EPOCH 8356: Cost = 5.000, x = 0.000
EPOCH 8357: Cost = 5.000, x = 0.000
EPOCH 8358: Cost = 5.000, x = 0.000
EPOCH 8359: Cost = 5.000, x = 0.000
EPOCH 8360: Cost = 5.000, x = 0.000
EPOCH 8361: Cost = 5.000, x = 0.000
EPOCH 8362: Cost = 5.000, x = 0.000
EPOCH 8363: Cost = 5.000, x = 0.000
EPOCH 8364: Cost = 5.000, x = 0.000
EPOCH 8365: Cost = 5.000, x = 0.000
EPOCH 8366: Cost = 5.000, x = 0.000
EPOCH 8367: Cost = 5.000, x = 0.000
EPOCH 8368: Cost = 5.000, x = 0.000
EPOCH 8369: Cost = 5.000, x = 0.000
EPOCH 8370: Cost = 5.000, x = 0.000
EPOCH 8371: Cost = 5.000, x = 0.000
EPOCH 8372: Cost = 5.000, x = 0.000
EPOCH 8373: Cost = 5.000, x = 0.000
EPOCH 8374: Cost = 5.000, x = 0.000
EPOCH 8375: Cost = 5.000, x = 0.000
EPOCH 8376: Cost = 5.000, x = 0.000
EPOCH 8377: Cost = 5.000, x 

#### Backpropagation Implementation

In [67]:
"""
Implement the backward method of the Sigmoid node.
"""
import numpy as np


class Node(object):
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes with edges into this node.
        self.inbound_nodes = inbound_nodes
        # The eventual value of this node. Set by running
        # the forward() method.
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        # Sets this node as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object.
        self.gradients = {self: 0}
        # Weights and bias may be inputs, so you need to sum
        # the gradient from output gradients.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] += grad_cost * 1


class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)


class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            """
            TODO: Your code goes here!

            Set the gradients property to the gradients with respect to each input.

            NOTE: See the Linear node and MSE node for examples.
            """
            self.gradients[self.inbound_nodes[0]] += self._sigmoid(self.inbound_nodes[0].value) * (1 - self._sigmoid(self.inbound_nodes[0].value)) * grad_cost


class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) ensures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calculates the gradient of the cost.

        This is the final node of the network so outbound nodes
        are not a concern.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()

In [68]:
"""
Test your network here!

No need to change this code, but feel free to tweak it
to test your network!

Make your changes to backward method of the Sigmoid class in miniflow.py
"""

import numpy as np

X, W, b = Input(), Input(), Input()
y = Input()
f = Linear(X, W, b)
a = Sigmoid(f)
cost = MSE(y, a)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2.], [3.]])
b_ = np.array([-3.])
y_ = np.array([1, 2])

feed_dict = {
    X: X_,
    y: y_,
    W: W_,
    b: b_,
}

graph = topological_sort(feed_dict)
forward_and_backward(graph)
# return the gradients for each Input
gradients = [t.gradients[t] for t in [X, y, W, b]]

"""
Expected output

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]
"""
print(gradients)

[array([[-3.34017280e-05, -5.01025919e-05],
       [-6.68040138e-05, -1.00206021e-04]]), array([[0.9999833],
       [1.9999833]]), array([[5.01028709e-05],
       [1.00205742e-04]]), array([-5.01028709e-05])]


#### Stochastic Gradient Descent to start learning on the above network
Stochastic Gradient Descent (SGD) is a version of Gradient Descent where on each forward pass a batch of data is randomly sampled from total dataset. Remember when we talked about the batch size earlier? That's the size of the batch. Ideally, the entire dataset would be fed into the neural network on each forward pass, but in practice, it's not practical due to memory constraints. SGD is an approximation of Gradient Descent, the more batches processed by the neural network, the better the approximation.

A naïve implementation of SGD involves:

Randomly sample a batch of data from the total dataset.
Running the network forward and backward to calculate the gradient (with data from (1)).
Apply the gradient descent update.
Repeat steps 1-3 until convergence or the loop is stopped by another mechanism (i.e. the number of epochs).
If all goes well, the network's loss should generally trend downwards, indicating more useful weights and biases over time.

So far, MiniFlow can already do step 2. In the following quiz, steps 1 and 4 are already implemented. It will be your job to implement step 3.

As a reminder, here's the gradient descent update equation, where $\alpha$ represents the learning rate:
x=x−$\alpha∗ 
∂cost/∂x
​	$

We're also going to use an actual dataset for this quiz, the [Boston Housing dataset](https://www.kaggle.com/c/boston-housing). After training the network will be able to predict prices of Boston housing!


![alt text](images/boston-back-bay-reflection.jpg "Boston House Price Prediction")

Each example in the dataset is a description of a house in the Boston suburbs, the description consists of 13 numerical values (features). Each example also has an associated price. With SGD, we're going to minimize the MSE between the actual price and the price predicted by the neural network based on the features.

In [3]:
import numpy as np


class Node(object):
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes with edges into this node.
        self.inbound_nodes = inbound_nodes
        # The eventual value of this node. Set by running
        # the forward() method.
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        # Sets this node as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object.
        self.gradients = {self: 0}
        # Weights and bias may be inputs, so you need to sum
        # the gradient from output gradients.
        for n in self.outbound_nodes:
            self.gradients[self] += n.gradients[self]

class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)


class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Sum the partial with respect to the input over all the outputs.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += sigmoid * (1 - sigmoid) * grad_cost


class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calculates the gradient of the cost.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()


def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # TODO: update all the `trainables` with SGD
    # You can access and assign the value of a trainable with `value` attribute.
    # Example:
    for t in trainables:
        t.value = t.value - learning_rate * t.gradients[t] 
    pass

In [5]:
"""
Check out the new network architecture and dataset!

Notice that the weights and biases are
generated randomly.

No need to change anything, but feel free to tweak
to test your network, play around with the epochs, batch size, etc!
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 10
# Total number of examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        forward_and_backward(graph)

        # Step 3
        sgd_update(trainables)

        loss += graph[-1].value

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))

Total number of examples = 506
Epoch: 1, Loss: 153.203
Epoch: 2, Loss: 28.424
Epoch: 3, Loss: 25.513
Epoch: 4, Loss: 24.838
Epoch: 5, Loss: 18.370
Epoch: 6, Loss: 18.032
Epoch: 7, Loss: 23.398
Epoch: 8, Loss: 18.770
Epoch: 9, Loss: 14.530
Epoch: 10, Loss: 15.318


### End of Book. Good job in completing NN from scratch!